7AiSR2

Eryk Pietrzak 240452

Bartłomiej Kacprzak 240417

In [ ]:
!pip install pulp

#!sudo apt-get install coinor-cbc glpk-utils coinor-clp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 11.7 MB/s eta 0:00:00


In [ ]:
import pulp

#print(pulp.___version___)

In [ ]:
from pulp import *

In [ ]:
"""
The Pracownik Problem
"""

pracownicy = ("Piotr", "Barbara", "Zenon")
dniTygodnia = ("poniedziałek","wtorek", "środa", "czwartek", "piątek", "sobota", "niedziela")

stawki = {"Piotr": 30,
         "Barbara": 50,
         "Zenon": 40}

maxGodzin = {"Piotr": 20,
         "Barbara": 80,
         "Zenon": 50}

#dni w jakich pracownicy pracują
dni = {"Piotr": ["poniedziałek","wtorek", "środa", "czwartek", "piątek"],
         "Barbara":  ["poniedziałek","wtorek", "środa", "czwartek", "piątek", "sobota", "niedziela"],
         "Zenon": ["poniedziałek", "wtorek", "środa", "sobota", "niedziela"]}

prob = LpProblem("ThePracownikProblem", LpMinimize)

harmonogramPiotr = [LpVariable("P"+ i, lowBound = 0, upBound = 10, cat='Integer') for i in dniTygodnia]
harmonogramBarbara = [LpVariable("B"+ i, lowBound = 0, upBound = 10, cat='Integer') for i in dniTygodnia]
harmonogramZenon = [LpVariable("Z"+ i, lowBound = 0, upBound = 10, cat='Integer') for i in dniTygodnia]
pracujePiotr = [LpVariable("_Czy_P_pracuje_w_" + i, cat='Binary') for i in dniTygodnia]
pracujeBarbara = [LpVariable("_Czy_B_pracuje_w_" + i, cat='Binary') for i in dniTygodnia]
pracujeZenon = [LpVariable("_Czy_Z_pracuje_w_" + i, cat='Binary') for i in dniTygodnia]

prob += lpSum(harmonogramPiotr) * stawki["Piotr"] + lpSum(harmonogramBarbara) * stawki["Barbara"] + lpSum(harmonogramZenon) * stawki["Zenon"], "koszt zatrudnienia"

for i in range(7):
  #10 godzin pracy w ciągu dnia
  prob += harmonogramPiotr[i] + harmonogramBarbara[i] + harmonogramZenon[i] == 10

  #praca tylko w dostępne dni
  if harmonogramPiotr[i].name[1:] not in dni["Piotr"]:
    prob += harmonogramPiotr[i] == 0
  if harmonogramBarbara[i].name[1:] not in dni["Barbara"]:
    prob += harmonogramBarbara[i] == 0
  if harmonogramZenon[i].name[1:] not in dni["Zenon"]:
    prob += harmonogramZenon[i] == 0

  #jeżeli pracuje to minimum 5 godzin
  prob += harmonogramPiotr[i] >= pracujePiotr[i] * 5
  prob += harmonogramBarbara[i] >= pracujeBarbara[i] * 5
  prob += harmonogramZenon[i] >= pracujeZenon[i] * 5

  #max 3 dni pracy w tygodniu
  prob += harmonogramPiotr[i] <= pracujePiotr[i] * 10
  prob += harmonogramBarbara[i] <= pracujeBarbara[i] * 10
  prob += harmonogramZenon[i] <= pracujeZenon[i] * 10

prob += lpSum(pracujePiotr) <= 3
prob += lpSum(pracujeBarbara) <= 3
prob += lpSum(pracujeZenon) <= 3

#tygodniowy limit godzin
prob += lpSum(harmonogramPiotr) <= maxGodzin["Piotr"]
prob += lpSum(harmonogramBarbara) <= maxGodzin["Barbara"]
prob += lpSum(harmonogramZenon) <= maxGodzin["Zenon"]

prob.writeLP("ThePracownikProblem.lp")
prob.solve()

#Wyśietlają się tylko niezerowe
print("Status:", LpStatus[prob.status])
for v in prob.variables():
    if v.varValue > 0:
      print(v.name, "=", v.varValue)

Status: Optimal
Bczwartek = 10.0
Bśroda = 10.0
Ppiątek = 10.0
Pwtorek = 10.0
Zniedziela = 10.0
Zponiedziałek = 10.0
Zsobota = 10.0
_Czy_B_pracuje_w_czwartek = 1.0
_Czy_B_pracuje_w_środa = 1.0
_Czy_P_pracuje_w_piątek = 1.0
_Czy_P_pracuje_w_wtorek = 1.0
_Czy_Z_pracuje_w_niedziela = 1.0
_Czy_Z_pracuje_w_poniedziałek = 1.0
_Czy_Z_pracuje_w_sobota = 1.0
